In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error,mean_absolute_error
import numpy.linalg as la
import math
from sklearn import preprocessing
from statsmodels.tsa.arima_model import ARIMA
import os

def preprocess_data(data, time_len, rate, seq_len, pre_len):
    data1 = np.mat(data)
    train_size = int(time_len * rate+5)
    train_data = data1[0:train_size]
    test_data = data1[train_size:time_len]
    return train_data, test_data


###### evaluation ######
def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    F_norm = la.norm(a-b)/la.norm(a)
    r2 = 1-((a-b)**2).sum()/((a-a.mean())**2).sum()
    var = 1-(np.var(a-b))/np.var(a)
    return rmse, mae, 1-F_norm, r2, var
def evaluation1(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    return rmse, mae
def evaluation2(a,b):
    F_norm = la.norm(a-b,'fro')/la.norm(a,'fro')
    r2 = 1-((a-b)**2).sum()/((a-a.mean())**2).sum()
    var = 1-(np.var(a-b))/np.var(a)
    return 1-F_norm, r2, var

path = r'data/3611817550_feature_matrix_X.xlsx'
data = pd.read_excel(path)

############## normalization ###############
price_frame = data
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1), copy=True)
data = min_max_scaler.fit_transform(price_frame)
data = pd.DataFrame(data)


time_len = data.shape[0]
num_nodes = data.shape[1]
train_rate = 0.8
seq_len = 5
pre_len = 1
train_data, test_data = preprocess_data(data, time_len, train_rate, seq_len, pre_len)

out = 'out/ARIMAtest'
path1 = 'precipitationARIMA_%r_seq%r_pre'%(seq_len,pre_len)
path = os.path.join(out,path1)
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
rng = pd.date_range('2022/8/25 13:00:00', periods=21026, freq='30min')
a1 = pd.DatetimeIndex(rng)
data.index = a1
num = data.shape[1]   
rmse,mae,acc,r2,var,pred,ori = [],[],[],[],[],[],[]
aic_values = {}
for i in range(30):
    ts = data.iloc[:,i]
    ts_log=np.log(ts)    
    ts_log=np.array(ts_log,dtype=np.float)
    where_are_inf = np.isinf(ts_log)
    ts_log[where_are_inf] = 0
    ts_log = pd.Series(ts_log)
    ts_log.index = a1
    
    
    for p in range(2):
        for q in range(2):
            try:
                model = ARIMA(train_data[:,i], order=(p, 0, q))
                result = model.fit()
                aic_values[(p, 0, q)] = result.aic
            except:
                continue
 
    min_aic = min(aic_values, key=aic_values.get)
    print("min aci:", min_aic)
    
    
    model = ARIMA(ts_log,order=min_aic)
    properModel = model.fit()
    predict_ts = properModel.predict(16825, dynamic=True)
    log_recover = np.exp(predict_ts)
    ts = ts[log_recover.index]
    er_rmse,er_mae,er_acc,r2_score,var_score = evaluation(ts,log_recover)
    
    pre = np.array(np.transpose(np.mat(log_recover)))
    pre = pre.repeat(pre_len ,axis=1)
    pred.append(pre)
    
    rmse.append(er_rmse)
    mae.append(er_mae)
    acc.append(er_acc)
    r2.append(r2_score)
    var.append(var_score)
    
print('arima_rmse:%r'%(np.mean(rmse)),
      'arima_mae:%r'%(np.mean(mae)),
      'arima_acc:%r'%(np.mean(acc)),
      'arima_r2:%r'%(np.mean(r2)),
      'arima_var:%r'%(np.mean(var)))
    
result1 = np.array(pred)
result1 = np.reshape(result1, [num_nodes,-1])
result1 = np.transpose(result1)
print(result1.shape)
testY1 = np.array(test_data)
testY1 = np.reshape(testY1, [-1,num_nodes])
print(testY1.shape)
############## renormalization ###############
test_label1 = min_max_scaler.inverse_transform(testY1)
test_output1 = min_max_scaler.inverse_transform(result1)

rmse1, mae1 = evaluation1(test_label1, test_output1)
#rmse1, mae1, acc1,r2,var = evaluation(testY1, result1)
acc1,r2,var = evaluation2(testY1, result1)


print('arima_rmse:%r'%rmse1,
      'arima_mae:%r'%mae1,
      'arima_acc:%r'%acc1,
      'arima_r2:%r'%r2,
      'arima_var:%r'%var)

result = pd.DataFrame(test_output1)
testYY = pd.DataFrame(test_label1)
result.to_csv(path+'/test_prediction.csv',index = False,header = False)
testYY.to_csv(path+'/test_true.csv',index = False,header = False)
evalution = []
evalution.append(rmse1)
evalution.append(mae1)
evalution.append(acc1)
evalution.append(r2)
evalution.append(var)
evalution = pd.DataFrame(evalution)
evalution.to_csv(path+'/evalution.csv',index=False,header=None)